<a href="https://colab.research.google.com/github/ChihChienLI/ChihChienLI/blob/main/MatchingScores_ipynb_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 78.2 MB/s eta 0:00:00


In [ ]:
from gurobipy import *
import pandas as pd

# read the data from excel
ms = pd.read_excel('MatchingScores.xlsx', index_col = 0)
print(ms)

        Tester  JavaDeveloper  Architect
Carlos      53             27         13
Joe         80             47         67
Monika      53             73         47


In [ ]:
# print the matching score of Joe as a Tester
#print(ms.loc['Joe','Tester'])
#print(ms.iloc[0,0])

# save the ms of JavaDeveloper job
ms_java = ms.iloc[:,1]
#print(ms_java)

# save the ms of Monika
ms_Monika = ms.iloc[-1,:]
#print(ms_Monika)

nr_resources = len(ms_java)
nr_jobs = len(ms_Monika)

In [ ]:
m = Model('MathcingScores')

# variables
x = m.addVars(nr_resources,nr_jobs, vtype = GRB.BINARY, name = 'x')

# objective function
m.setObjective( quicksum( ms.iloc[i,j]*x[i,j] for i in range(nr_resources) for j in range(nr_jobs)), GRB.MAXIMIZE )

# constraints 
m.addConstrs( quicksum( x[i,j] for i in range(nr_resources)) == 1 for j in range(nr_jobs))
m.addConstrs( quicksum( x[i,j] for j in range(nr_jobs)) <= 1 for i in range(nr_resources) )

# solve the model
m.optimize()

# print the solution and the optimal objective function value
for v in m.getVars():
  print(v.varName, v.X)

print('Optimal total matching score: ', m.objval)

In [ ]:
# store the solution in a file
f = open('MS_solution.txt', 'w')
for v in m.getVars():
  f.write( str(v.varName)+" = "+str(v.X)+"\n")

f.write("Optimal total matching score: "+str(m.objval))
f.close()